In [1]:
import os
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- Seed for reproducibility ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("✅ Imports & seed initialized!")

✅ Imports & seed initialized!


In [2]:
# Check if dataset file exists after download
print("data.zip downloaded:", os.path.exists("data.zip"))

data.zip downloaded: True


In [3]:
import zipfile

with zipfile.ZipFile("data.zip", "r") as z:
    z.extractall(".")  # extract in workspace root

print("✅ Dataset extracted!")

✅ Dataset extracted!


In [4]:
print("Train dir exists:", os.path.exists("train"))
print("Test dir exists:", os.path.exists("test"))

# See example files
if os.path.exists("train"):
    print("\nSample train folder content:", os.listdir("train")[:5])
if os.path.exists("test"):
    print("Sample test folder content:", os.listdir("test")[:5])

Train dir exists: False
Test dir exists: False


datasets → loads image datasets

transforms → prepares images (resize, convert to tensor, normalize)

DataLoader → feeds images to the model in batches

Set test_transforms = train_transforms → meaning both train and test images get the same preparation. 
    
Dataset loading - Reads images from those folders. Automatically assigns labels based on folder names (curly, straight). Applies the transforms above.

Creating loaders - 20 images at a time go into model (batch_size=20), shuffle=True on train → random order each epoch (good for training)
shuffle=False on test → fixed order (correct, required)


In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Image transformations required by homework
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = train_transforms  # same transforms, no extra preprocessing needed

# Load dataset from extracted folder
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
test_dataset = datasets.ImageFolder("data/test", transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

print("✅ DataLoaders created!")
print("Classes detected:", train_dataset.classes)
print("Number of train images:", len(train_dataset))
print("Number of test images:", len(test_dataset))


✅ DataLoaders created!
Classes detected: ['curly', 'straight']
Number of train images: 800
Number of test images: 201


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cpu


In [9]:
import torch.nn as nn

class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # First conv layer: input channel 3 (RGB), output channel 32, kernel 3×3
        self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2))

        # After conv + pool, we flatten and pass to linear layer with 64 neurons
        self.fc1 = nn.Linear(32 * 99 * 99, 64)  # 99*99 comes after 3×3 conv with no padding + 2×2 pool
        self.relu2 = nn.ReLU()

        # Final output layer for binary classification (1 neuron)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # activation for binary classification

    def forward(self, x):
        x = self.pool(self.relu(self.conv(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.relu2(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize model
model = HairTypeCNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()

print("✅ CNN model initialized!")
print("Total parameters:", sum(p.numel() for p in model.parameters()))


✅ CNN model initialized!
Total parameters: 20073473


A CNN learns images by scanning with filters, extracting strong patterns, shrinking the data, flattening it into numbers, 
and using linear layers to learn how patterns map to a yes/no decision — in this case, curly vs straight hair.

1. nn.BCEWithLogitsLoss()
2. Total parameters: 20073473

In [10]:
# Validation transformations (same as train/test for consistency)
val_transforms = train_transforms

# Load validation dataset (if you want a separate validation split, we can create one slowly after)
validation_dataset = datasets.ImageFolder("data/test", transform=val_transforms)

# Create DataLoader for validation
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=True)

print("✅ Validation DataLoader created!")
print("Validation images:", len(validation_dataset))


✅ Validation DataLoader created!
Validation images: 201


In [11]:
num_epochs = 10
history = {'acc': [], 'loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6736, Acc: 0.4875
Epoch 2/10, Loss: 0.6541, Acc: 0.4875
Epoch 3/10, Loss: 0.6376, Acc: 0.4875
Epoch 4/10, Loss: 0.6345, Acc: 0.4875
Epoch 5/10, Loss: 0.6220, Acc: 0.4875
Epoch 6/10, Loss: 0.6177, Acc: 0.4875
Epoch 7/10, Loss: 0.6130, Acc: 0.4875
Epoch 8/10, Loss: 0.6087, Acc: 0.4875
Epoch 9/10, Loss: 0.6044, Acc: 0.4875
Epoch 10/10, Loss: 0.5948, Acc: 0.4875


In [12]:
print("Accuracy history:", history['acc'])
print("Loss history:", history['loss'])

Accuracy history: [0.4875, 0.4875, 0.4875, 0.4875, 0.4875, 0.4875, 0.4875, 0.4875, 0.4875, 0.4875]
Loss history: [0.6736191228032112, 0.654145672917366, 0.6376249462366104, 0.6345452621579171, 0.622015492618084, 0.6177252188324929, 0.6130436688661576, 0.6086574584245682, 0.6043648511171341, 0.5948178455233574]


median is 0.4875
std 0.007

In [13]:
import numpy as np

# --- Question 3: median training accuracy ---
median_acc = float(np.median(history['acc']))
print("Median training accuracy:", median_acc)

# --- Question 4: standard deviation of training loss ---
std_loss = float(np.std(history['loss']))
print("Std of training loss:", std_loss)

Median training accuracy: 0.4875
Std of training loss: 0.022937467873778965


In [14]:
from torchvision import transforms

# Updated train transforms with augmentations
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ Train transforms updated with augmentation!")


✅ Train transforms updated with augmentation!


What this does
Still resizes to 200×200
Adds:
Random rotations
Random crops (slight zoom in/out)
Horizontal flips (mirror)
Keeps normalization and tensor conversion

load only the training data again with the new transform

In [15]:
from torchvision import datasets
from torch.utils.data import DataLoader

# Reload only training dataset with augmentations
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)

# Recreate DataLoader to use updated transforms
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

print("✅ Training dataset and loader updated with augmentations!")
print("Total train images:", len(train_dataset))


✅ Training dataset and loader updated with augmentations!
Total train images: 800


In [16]:
num_epochs = 10  # 10 more epochs
test_loss_history = []
test_acc_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Evaluate on test set
    model.eval()
    test_running_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    epoch_test_loss = test_running_loss / test_total
    epoch_test_acc = test_correct / test_total

    test_loss_history.append(epoch_test_loss)
    test_acc_history.append(epoch_test_acc)

    print(f"Epoch {epoch+1}/10 (Augmented), ",
          f"Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_acc:.4f}")


Epoch 1/10 (Augmented),  Test Loss: 0.6560, Test Acc: 0.4975
Epoch 2/10 (Augmented),  Test Loss: 0.6564, Test Acc: 0.5075
Epoch 3/10 (Augmented),  Test Loss: 0.6529, Test Acc: 0.5373
Epoch 4/10 (Augmented),  Test Loss: 0.6535, Test Acc: 0.5124
Epoch 5/10 (Augmented),  Test Loss: 0.6493, Test Acc: 0.5124
Epoch 6/10 (Augmented),  Test Loss: 0.6489, Test Acc: 0.5174
Epoch 7/10 (Augmented),  Test Loss: 0.6485, Test Acc: 0.5522
Epoch 8/10 (Augmented),  Test Loss: 0.6493, Test Acc: 0.5274
Epoch 9/10 (Augmented),  Test Loss: 0.6522, Test Acc: 0.5224
Epoch 10/10 (Augmented),  Test Loss: 0.6548, Test Acc: 0.5473


In [17]:
import numpy as np

# Results your training already produced (Augmented test performance)
test_loss_history = [
    0.6560, 0.6564, 0.6529, 0.6535, 0.6493,
    0.6489, 0.6485, 0.6493, 0.6522, 0.6548
]

test_acc_history = [
    0.4975, 0.5075, 0.5373, 0.5124, 0.5124,
    0.5174, 0.5522, 0.5274, 0.5224, 0.5473
]

# --- Question 5: Mean Test Loss ---
mean_test_loss = float(np.mean(test_loss_history))
print("Q5 → Mean of test loss:", mean_test_loss)

# --- Question 6: Avg Test Accuracy for Last 5 Epochs (6–10) ---
last_5_test_acc = test_acc_history[5:]  # epochs 6-10
avg_last_5_test_acc = float(np.mean(last_5_test_acc))
print("Q6 → Avg test accuracy for last 5 epochs:", avg_last_5_test_acc)


Q5 → Mean of test loss: 0.65218
Q6 → Avg test accuracy for last 5 epochs: 0.5333399999999999


This model performs slightly better after augmentations, but still near-random accuracy (~53%), 
so improvement exists but needs more epochs/architecture work in future to match top submissions